# Self-Organizing AI-ML Visualization Demo

This notebook demonstrates the various visualization capabilities of the simulation framework.

In [ ]:
import sys
import os
sys.path.append('..')

import asyncio
import torch
import numpy as np
from src.simulations.prototypes.self_organizing_sim import SelfOrganizingSimulation, SimulationConfig
from src.simulations.utils.visualization import (
    create_2d_scatter,
    create_3d_visualization,
    plot_statistics,
    create_phase_space_plot
)
import plotly.io as pio
pio.renderers.default = 'notebook'

print(f"Using device: {'cuda' if torch.cuda.is_available() else 'cpu'}")

## 1. Initialize and Run Simulation

In [ ]:
# Configure simulation
config = SimulationConfig(
    batch_size=64,
    max_agents=100,  # Reduced for visualization clarity
    learning_rate=0.001,
    max_iterations=1000
)

# Create simulation instance
sim = SelfOrganizingSimulation(config)

# Run simulation
async def run_sim():
    return await sim.run_simulation(num_steps=200)

results = await run_sim()
print("Simulation completed!")

## 2. 2D Real-time Visualization

In [ ]:
# Create 2D scatter plot of final state
fig_2d = create_2d_scatter(results['final_state'], frame_idx=200)
fig_2d.show()

## 3. 3D Trajectory Visualization

In [ ]:
# Extract states history
states_history = []
async def collect_states():
    state = await sim.initialize()
    states_history.append(state)
    for _ in range(100):
        state = await sim.step(state)
        states_history.append(state)
    return states_history

states_history = await collect_states()

# Create 3D visualization
fig_3d = create_3d_visualization(states_history, skip_frames=2)
fig_3d.show()

## 4. Statistical Analysis

In [ ]:
# Plot statistics over time
fig_stats = plot_statistics(results['stats_history'])
fig_stats.show()

## 5. Phase Space Analysis

In [ ]:
# Create phase space plot
fig_phase = create_phase_space_plot(results['final_state'])
fig_phase.show()

## 6. Interactive Animation

In [ ]:
import plotly.graph_objects as go
from ipywidgets import interact

@interact(frame=(0, len(states_history)-1))
def update_frame(frame):
    fig = create_2d_scatter(states_history[frame], frame_idx=frame)
    fig.show()

## 7. Energy Analysis

In [ ]:
# Calculate and plot system energy over time
energies = [stats['energy'] for stats in results['stats_history']]
fig = go.Figure()
fig.add_trace(go.Scatter(y=energies, mode='lines', name='System Energy'))
fig.update_layout(title='System Energy Over Time',
                  xaxis_title='Time Step',
                  yaxis_title='Energy')
fig.show()